In [1]:
import boto3
import numpy as np
import time

from skimage import io
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.mxnet import MXNet

# Creating a model using MXNet

## Configure hyperparameters

In [2]:
# Number of output classes
num_classes = 2

# Batch size for training
mini_batch_size =  128

# Max epochs for training
epochs = 2

# Learning rate
learning_rate = 0.01

## Create a unique job name 

In [18]:
job_name_prefix = 'breast-cancer-detection'
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
job_name = job_name_prefix + timestamp

## Specify the input paths for the job

In [4]:
bucket = 'sagemaker-data-jv'
input_prefix = 'breast-cancer-detection/input/recordio'
input_train = 's3://{}/{}/train/'.format(bucket, input_prefix)
input_test = 's3://{}/{}/test/'.format(bucket, input_prefix)

## Specify the output path for the job

In [5]:
output_prefix = 'breast-cancer-detection/output'
output_path = 's3://{}/{}/'.format(bucket, output_prefix)

## Configure training instances

In [6]:
instance_count = 1
instance_type = 'ml.p2.xlarge'
volume_size_gb = 50

## Get the execution role

In [7]:
role = get_execution_role()

## Configure train timeout

In [8]:
train_timeout = 360000

## Specify the path to the training script

In [9]:
training_script_path = 'mxnetScript.py'

## Create a sagemaker.MXNet estimator

In [19]:
estimator = MXNet(entry_point=training_script_path,
                  role=role,
                  train_instance_count=instance_count,
                  train_instance_type=instance_type,
                  train_volume_size=volume_size_gb,
                  train_max_run=train_timeout,
                  output_path=output_path,
                  framework_version='1.3.0',
                  py_version = 'py3',
                  hyperparameters = {
                    'num-classes': num_classes,
                    'mini-batch-size': mini_batch_size,
                    'epochs': epochs,
                    'learning-rate': learning_rate
                  })